In [1]:
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the number of classes in the dataset
num_classes = 8

# Define the input size of the Xception model
input_shape = (299, 299, 3)

# Load the pre-trained Xception model without the top layer
base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

# Add a new top layer for our specific classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create a new model with our custom top layer
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

# Define the data directories
train_dir = './Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
val_dir = './Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'

# Define the data generators with data augmentation for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=input_shape[:2], batch_size=32, class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(val_dir, target_size=input_shape[:2], batch_size=32, class_mode='categorical')

# Compile the model with appropriate loss function, optimizer and metrics
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on our dataset
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Unfreeze the weights of the pre-trained layers after the model has been trained on our specific task
for layer in base_model.layers:
    layer.trainable = True

# Re-compile the model with a lower learning rate to fine-tune the pre-trained layers
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model on our dataset
model.fit(train_generator, epochs=10, validation_data=val_generator)


NotADirectoryError: [WinError 267] The directory name is invalid: './Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'

In [2]:
def load_features(photos):
    #loading all features
    all_features = load(open("features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features

In [6]:
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos

filename = "./Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt"

#train = loading_data(filename)
train_imgs = load_photos(filename)
train_features = load_features(train_imgs)

NameError: name 'load_doc' is not defined